In [286]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.ticker as ticker
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
plt.rcParams.update({'font.size': 12})
import seaborn as sns
import cmath
import math 
def custom_plot_single(x_lst, y_lst, label_lst, xlim, ylim, label,pltname,
                       color=['k','r','b','g','o','br'],
                       linestyle=['solid','dashed','solid','dashed','solid','dashed'],
                       markertype=[None,None,'o','^','o','^'],
                       fillstyle=['none','none','none','none','full','full'],
                       linewidth=20*[3],
                       markevery=[45,30,50,40,56,72,63,95]):
    
    fig, ax1 = plt.subplots(1,1)
    fig.set_size_inches(10,10)
    fig.patch.set_facecolor('white')
    ax1.patch.set_facecolor('white')
    for p in range(0, len(x_lst)):
        ax1.plot(x_lst[p], y_lst[p], color[p],
                 linewidth=linewidth[p],
                 linestyle=linestyle[p],
                 marker=markertype[p],
                 fillstyle=fillstyle[p],
                 markevery=markevery[p],
                 markersize=8,
                 label=label_lst[p])

    ax1.legend(prop={'size': 20},loc='best')
    ax1.tick_params(which='minor', width=2, length=4, color='k')
    ax1.tick_params(which='major', width=2, length=8, color='k')

    ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator())
    ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator())
    ax1.legend(prop={'size': 20},loc='best')#,shadow=True)

    ax1.set_ylim(ylim[0],ylim[1])
    ax1.set_xlim(xlim[0],xlim[1])
    ax1.set_xlabel(label[0], fontsize=25)#, fontdict=dict(weight='bold'))
    ax1.set_ylabel(label[1], fontsize=25)#, fontdict=dict(weight='bold'))
    fontsize=25
    for tick in ax1.xaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)

    for tick in ax1.yaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
        
    plt.savefig(pltname, bbox_inches = "tight")

In [287]:
PrintFigures=False

In [288]:
num_CNT_Files=1
num_layers=280
num_layers_contacts=28
num_layers_ch =num_layers-2*num_layers_contacts #ch for channel

acc = 0.142e-9 #nm
nring_per_unitcell = 4 #for a zigzag nanotube
Y_offset = -15.0165e-9 #nm
nano=1e-9

layer = np.empty((num_CNT_Files,num_layers),dtype=int)
avg_field = np.empty((num_CNT_Files,num_layers),dtype=float)

file_name = np.empty(num_CNT_Files,dtype=object)

#path = '../ImpRuns/GVdot5V_NoChargeDensity/'
#path = '../ImpRuns/GVdot5V_someChargeDensity/'
#path = '../ImpRuns/Verification_XdZdot5V_NoChargeDensity/'
path = ''

In [289]:
for c in range(0, num_CNT_Files):
    file_name[c] = path + 'avg_gatherField_%d.dat'%(c)

for f in range(0,num_CNT_Files):
    file = open(file_name[f])
    lst = []
    counter=0
    for line in file:     
        if (counter >= 0):
            lst += [line.split()]
        else:
            print(line)
        counter = counter + 1 
    layer[f]      = [x[0] for x in lst]
    avg_field[f]  = [x[1] for x in lst]
        

In [290]:
print(file_name[0])

avg_gatherField_0.dat


In [291]:
axial_loc = np.empty(num_layers,dtype=float)

def get_axial_loc(r,ACC):
    y=0.
    if(r % 2 == 0):
        y = ACC
    elif (r%2 == 1):
        y = ACC/2
    return y
 
axial_loc[0]=acc

for l in range(1,num_layers):
    rID_in_unitcell = l%nring_per_unitcell   
#    print(rID_in_unitcell)
    axial_loc[l] = axial_loc[l-1] + get_axial_loc(rID_in_unitcell,acc)
    
for l in range(0,num_layers):
    axial_loc[l] += Y_offset                  
    

In [292]:
layer_ch = np.empty((num_CNT_Files,num_layers_ch),dtype=int)

axial_loc_ch = np.empty(num_layers_ch,dtype=float)
avg_field_ch = np.empty((num_CNT_Files,num_layers_ch),dtype=float)

for f in range(0,num_CNT_Files):
    axial_loc_ch = axial_loc[num_layers_contacts:num_layers-num_layers_contacts]
    avg_field_ch[f] = avg_field[f][num_layers_contacts:num_layers-num_layers_contacts]
    
# custom_plot_single([axial_loc/nano,axial_loc_ch/nano], 
#                    [avg_field[0],avg_field_ch[0]],
#                    [r'avg field',r'avg field (in channel)'],
#                    [-16,16],[-0.01,0.5],[r'Layer / (nm)',r'Potential / (V)'],'Layer_vs_Potential.png')

if(PrintFigures):
    custom_plot_single([axial_loc/nano], 
                       [avg_field[0]],
                       [r'avg field',r'avg field (in channel)'],
                       [-16,16],[-0.01,0.5],[r'y / (nm)',r'Avg. potential on CNT surface / (V)'],'Layer_vs_Potential.png')



# NEGF

### parameter definition

In [318]:
h=1.06e-34
q=1.602e-19
G_quantum=2*q**2/h #at zero temperature
print('G_quantum',G_quantum)
print('R_quantum',1/G_quantum)
gamma=2.5 #eV
acc=0.142e-9
m=17
n=0
nano=1e-9

#N=num_layers_ch
#U=avg_field_ch

def get_radius(acc,m,n):
    return acc*(np.sqrt(3.)/(2*np.pi))*np.sqrt(m**2 + m*n + n**2)


R = get_radius(acc,m,n)
print('Radius of CNT (nm):', R/nano)

Eg_min = acc*gamma/R
print('bandgap, H/(eV):', Eg_min)

Us=0.0 #V, Source potential
Ud=0.0 #V, Drain potential

N=4
U=np.zeros(N)
print(U)
print('number of layers in the channel: ', N)

q = math.gcd(m, n)
Int=(m-n)/(3*q)
print('q',q)
print('Int',Int)
if(Int.is_integer()):
    d_R = 3*q
else:
    d_R = q
print('d_R',d_R)
Natoms_per_UC=int(4*(m**2+m*n+n**2)/d_R)
print('number of atoms per unit cell',Natoms_per_UC)

if(n==0): #zigzag
    M =  int(Natoms_per_UC/4)
elif(m==n): #armchair
    M = int(Natoms_per_UC/2)
else:
    M = np.abs(m-n)
print('number of total modes or atoms along the ring', M)
M_red = M
print('number of modes used to construct hamiltonian', M_red)


G_quantum 0.00048422716981132076
R_quantum 2065.1464071907617
Radius of CNT (nm): 0.665453976774103
bandgap, H/(eV): 0.5334704012453583
[0. 0. 0. 0.]
number of layers in the channel:  4
q 17
Int 0.3333333333333333
d_R 17
number of atoms per unit cell 68
number of total modes or atoms along the ring 17
number of modes used to construct hamiltonian 17


### Bandgap (From Theory)

In [294]:

def get_bandgap_for_mode(p,acc,gamma,m,n):
    R=get_radius(acc,m,n)
    Eg_case=np.zeros(6,dtype=float)
    if(p>0):
        Eg_case[0] = (acc*gamma/R)*np.abs(3*p - (2*m+n))
        Eg_case[1] = (acc*gamma/R)*np.abs(3*p - (m+2*n))
        Eg_case[2] = (acc*gamma/R)*np.abs(3*p - (m-n))
        Eg_case[3] = (acc*gamma/R)*np.abs(-3*p + (2*m+n))
        Eg_case[4] = (acc*gamma/R)*np.abs(-3*p + (m+2*n))
        Eg_case[5] = (acc*gamma/R)*np.abs(3*p - (m-n))
    elif(p<0):
        Eg_case[0] = (acc*gamma/R)*np.abs(-3*p - (2*m+n))
        Eg_case[1] = (acc*gamma/R)*np.abs(-3*p - (m+2*n))
        Eg_case[2] = (acc*gamma/R)*np.abs(-3*p - (m-n))
        Eg_case[3] = (acc*gamma/R)*np.abs(3*p + (2*m+n))
        Eg_case[4] = (acc*gamma/R)*np.abs(3*p + (m+2*n))
        Eg_case[5] = (acc*gamma/R)*np.abs(-3*p - (m-n))
     
    Eg = np.amin(Eg_case)
    min_index = np.argmin(Eg_case)
    return (Eg,min_index+1)


m_arranged=np.arange(1,M+1,1)
m_arranged_rev=np.zeros(np.size(m_arranged),dtype=int)
counter=0
for i in reversed(m_arranged):
    m_arranged_rev[counter] = -i
    counter=counter+1
M_arr = np.concatenate((m_arranged_rev,m_arranged))
#M_arr = np.arange(-m+1,m,1)

print('M_arr',M_arr)

Eg_arr = np.zeros(M, dtype=float)
counter=0
for p in range(1,M+1):
    eg, case = get_bandgap_for_mode(p,acc,gamma,m,n)
    Eg_arr[p-1]=eg
    print('mode, bandgap', p , Eg_arr[p-1], case)
    #counter +=1

Eg_min = np.amin(Eg_arr)
print('minimum bandgap, acc*gamma/R',Eg_min, acc*gamma/R)

Eg_min_index_arr = np.where(Eg_arr==Eg_arr.min())
print('band with minimum bandgap:')
for j in Eg_min_index_arr:
    print(M_arr[M+j])
    
sorted_M_arr = [x for _,x in sorted(zip(Eg_arr,np.arange(1,M+1,1)))] 
#print('sorted_M_arr:',sorted_M_arr)
print('size of sorted_M_arr:',np.shape(sorted_M_arr))

M_red_arr = np.zeros(M_red,dtype=int)

for i in range(0,M_red):
    M_red_arr[i] = sorted_M_arr[i]
    
print('M_red_arr',M_red_arr)

M_arr [-17 -16 -15 -14 -13 -12 -11 -10  -9  -8  -7  -6  -5  -4  -3  -2  -1   1
   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17]
mode, bandgap 1 7.468585617435016 2
mode, bandgap 2 5.868174413698941 2
mode, bandgap 3 4.267763209962866 2
mode, bandgap 4 2.6673520062267913 2
mode, bandgap 5 1.0669408024907165 2
mode, bandgap 6 0.5334704012453583 2
mode, bandgap 7 2.133881604981433 2
mode, bandgap 8 3.734292808717508 2
mode, bandgap 9 3.734292808717508 1
mode, bandgap 10 2.133881604981433 1
mode, bandgap 11 0.5334704012453583 1
mode, bandgap 12 1.0669408024907165 1
mode, bandgap 13 2.6673520062267913 1
mode, bandgap 14 4.267763209962866 1
mode, bandgap 15 5.868174413698941 1
mode, bandgap 16 7.468585617435016 1
mode, bandgap 17 9.06899682117109 1
minimum bandgap, acc*gamma/R 0.5334704012453583 0.5334704012453583
band with minimum bandgap:
[ 6 11]
size of sorted_M_arr: (17,)
M_red_arr [ 6 11  5 12  7 10  4 13  8  9  3 14  2 15  1 16 17]


### Band structure and First Brillouin Zone


In [295]:
def get_dispersion(m,gamma,nu,E_nu,kxa_arr):
    kyb=2*np.pi*nu/(2*m)
    for kxa in kxa_arr:
        E_nu.append(gamma*np.sqrt(1+(4*np.cos(kyb)*np.cos(kxa))+(4*np.cos(kyb)**2)))

k_pi=np.linspace(-np.pi,np.pi,2401) # range*pi    
k = k_pi/np.pi

nu_label=str(sorted_M_arr)
#print(nu_label)
E_k=[]
linewidth=[]
counter=0
lw=3


for p in sorted_M_arr:
    E_nu=[]
    get_dispersion(m,gamma,p,E_nu,k_pi)
    E_k.append(E_nu)
    E_nu_inv = []
    for i in range(0,len(k)):
        E_nu_inv.append(-1*E_nu[i])
    E_k.append(E_nu_inv)
    linewidth.append(lw)
    linewidth.append(lw)
    counter += 1
    if (counter%2 == 0):
        lw=4*np.exp(-0.2*counter)

print('size of E_k',np.shape(E_k))

if(PrintFigures):
    custom_plot_single(2*M*[k], 
                       E_k,
                       2*M*[None],
                       [-1,1],[-8,8],[r'k$_x$ a/$\pi$',r'Energy, E / (eV)'],'Dispersion_all.png',
                       2*M*['k'],
                       2*M*['solid'],
                       2*M*[None],
                       2*M*['none'],
                       linewidth,2*M*['none'])

size of E_k (34, 2401)


In [296]:
import operator
p1=sorted_M_arr[0]
p2=sorted_M_arr[1]
print('p2',p2)

Eg = gamma*acc/R
print('gamma',gamma,acc,R)
a=3*acc/2 
b=np.sqrt(3)*acc/2

def get_linear_approximate_dispersion(m,gamma,nu,E_nu,kxa_arr,kf):
    kyb=2*np.pi*nu/(2*m)
    factor=-1
    for kxa in kxa_arr:
        if(kxa > 0):
            factor =1
        E_nu.append((3*gamma/2)*np.sqrt((kxa*2/3-factor*kf[0]*acc)**2 + ((kyb*2/np.sqrt(3)-kf[1]*acc))**2))      

kf=[np.pi/a, np.pi/(3*b)]
E_lin_1 = []
get_linear_approximate_dispersion(m,gamma,p1,E_lin_1,k*np.pi,kf)
E_lin_1_minus=list(map(operator.mul, E_lin_1, [-1]*len(E_lin_1))) #multiply all elements in list by -1

kf=[0, 2*np.pi/(3*b)]
E_lin_2 = []
get_linear_approximate_dispersion(m,gamma,p2,E_lin_2,k*np.pi,kf)
E_lin_2_minus=list(map(operator.mul, E_lin_2, [-1]*len(E_lin_2))) #multiply all elements in list by -1

if(PrintFigures):
    custom_plot_single([k,k,k,k,k,k,k,k], 
                       [E_k[2*1-2],E_k[2*1-1],E_k[2*2-2],E_k[2*2-1],
                        E_lin_1,E_lin_1_minus,E_lin_2, E_lin_2_minus],
                       [r'$\nu$='+str(int(p1)),None, r'$\nu$='+str(int(p2)), None, r'$\nu$='+str(int(p1))+' (linear)',None, r'$\nu$='+str(int(p2))+' (linear)',None],
                       [-1,1],[-8,8],[r'k$_x$ a/$\pi$',r'Energy, E / (eV)'],'Dispersion.png',
                       ['k','k','b','b','r','r','g','g'],
                       ['solid','solid','solid','solid','dashed','dashed','dashed','dashed'],
                       8*[None],
                       8*['none'])
print('minimum band gap: (eV)', Eg)

p2 11
gamma 2.5 1.42e-10 6.65453976774103e-10
minimum band gap: (eV) 0.5334704012453583


### Construct hamiltonian H

In [297]:
#The following 2 functions are to create off-diagonal elements of a hamiltonian
def get_zeros_and_ones(L,lst):
    for i in range(0,L):
        if(i%2==0):
            lst.append(0)
        else:
            lst.append(1)
            
def get_ones_and_zeros(L, lst): 
    for i in range(0,L):
        if(i%2==0):
            lst.append(1)
        else:
            lst.append(0)

lst_01 = [] 
lst_10 = [] 

get_zeros_and_ones(N-1, lst_01)
get_ones_and_zeros(N-1, lst_10)

def get_beta(gamma, M, J): #overlap coefficient in mode-space approximation
    return 2*gamma*np.cos(np.pi*J/M) #*np.exp(-1j*np.pi*J/M)

In [298]:
H_mode=np.zeros((M_red,N,N),dtype=float)
print(np.shape(H_mode))
zero_mat=np.zeros((N,N),dtype=float)
#print(np.shape(zero_mat))
H=np.zeros((M_red*N,M_red*N),dtype=float)
print(np.shape(H))
beta = np.zeros(M_red,dtype=float)

for i in range(0,M_red):
    p=M_red_arr[i]
    beta[i] = get_beta(gamma,M,p)
    H_mode[i] =U*np.diag(np.ones(N)) + gamma*np.diag(lst_01,-1)  + beta[i]*np.diag(lst_10,-1) \
    + gamma*np.diag(lst_01,1) + beta[i]*np.diag(lst_10,1) 

H_lst=[]
for i in range (0, M_red):
    submat = []
    for j in range(0, M_red):
        if j==i:
            submat.append(H_mode[i])
        else:
            submat.append(zero_mat)          
    H_lst.append(submat)
H=np.block(H_lst)

if(PrintFigures):
    plt.figure(figsize = (16,16))
    sns.heatmap(H,cmap='coolwarm',annot=True)

(17, 4, 4)
(68, 68)


### define E-space


In [299]:
Emin=-8.001 #eV
Emax=8.001 #eV
Epts=40000

E=np.linspace(Emin,Emax,Epts,dtype=float)
#print('E',E)

#Define fermi energies and potential due to bias
Ef=0
kb=8.617333262e-5 #eV/K
Temp=298 #K
print(kb*Temp)
mu_s=Ef + Us
mu_d=Ef + Ud 
mu_c = Ef
f_s=1./(1.+np.exp((E-mu_s)/(kb*Temp)))
f_d=1./(1.+np.exp((E-mu_d)/(kb*Temp)))
f_c=1./(1.+np.exp((E-mu_c)/(kb*Temp)))
print(mu_s, mu_d, mu_c)

dfdE_s = np.gradient(f_s)
dfdE_d = np.gradient(f_d)
dfdE_c = np.gradient(f_c)

0.02567965312076
0.0 0.0 0


In [300]:
if(PrintFigures):
    custom_plot_single([E-Ef,E-Ef,E-Ef], 
                       [ f_s,f_d,f_c],
                       [r'source (%.2f V)'%(Us),r'drain (%.2f V)'%(Ud),r'channel (%.f K)'%(Temp)],
                       [-0.5,0.5],[0,1],[r'Energy, E / (eV)',r'Fermi function, f'],'fermi_functions.png',
                       ['k','r','g'],
                       ['solid','dashed','solid','dashed'],
                       ['o','^','o','^'],
                       4*['none'],
                       [2,2,2,2])

    custom_plot_single([E-Ef], 
                       [-dfdE_c ],
                       [r'channel (%.f K)'%(Temp)],
                       [-0.5,0.5],[0,0.005],[r'Energy, E / (eV)',r'Gradient of Fermi function, -$\frac{\partial{f}}{\partial{E}}$'],'gradient_of_fermi_functions.png',
                       ['k','r','k','r'],
                       ['solid','dashed','solid','dashed'],
                       ['o','^','o','^'],
                       4*['none'],
                       [2,2,2,2])


### compute self-energy


In [301]:
def get_analytical_retarted_surface_GF(E, Us,gamma,beta_j,Print):
    EmUs = E-Us
    Factor=EmUs**2 + gamma**2 - beta_j**2
    Sqrt=cmath.sqrt(Factor**2 - 4 * EmUs**2 * gamma**2)
    Denom = 2 * gamma**2 * EmUs

    Numer1= (Factor + Sqrt)
    Numer2= (Factor - Sqrt)
  
    if(Print):
        print('E,Us,gamma,beta_j',E,Us,gamma,beta_j)
        print('*',4 * EmUs**2 * gamma**2/Factor**2)
        print('Sqrt',Sqrt)
        print('Denom',Denom)
        print('Numer1',Numer1)
        print('Numer2',Numer2)
        print(np.imag(Numer1),np.imag(Numer2))

    zplus=1j*1e-5 #small imaginary number

    val1 = Numer1/Denom - zplus
    val2 = Numer2/Denom - zplus
    
    val = 0. + 1j*0.
    if(np.imag(val1) < 0.):
        val = val1 
    elif (np.imag(val2) < 0.):
        val = val2 
        
    if(Print): print(val)
#     if(np.abs(np.imag(val)) < 1e-2):
#         val = -zplus
    return val

In [302]:
gR_surf_s1 = np.zeros((M_red,Epts),dtype=complex)
gR_surf_d1 = np.zeros((M_red,Epts),dtype=complex)
Ud=0
print('Us,Ud',Us,Ud)

for j in range(0,M_red):
    Print=False
    for e in range(0,Epts):
        #if(j==10):
        #    Print=True
        gR_surf_s1[j][e] = get_analytical_retarted_surface_GF(E[e],Us,gamma,beta[j],Print)
        gR_surf_d1[j][e] = get_analytical_retarted_surface_GF(E[e],Ud,gamma,beta[j],Print)


Us,Ud 0.0 0


In [303]:
if(PrintFigures):
    custom_plot_single([E,E], 
                       [ np.imag(gR_surf_s1[0]),np.imag(gR_surf_d1[0]) ],
                       [r'$g_1$, source ('+str(Us)+' V)',r'$g_1$, drain ('+str(Ud)+' V)'],
                       [-9,9],[-0.45,0.1],[r'Energy, E / (eV)',r'Surface Green\'s function, $g_1$'],'g1.png',
                       ['r','r','b','b'],
                       ['solid','dashed','solid','dashed'],
                       4*[None],#['o','^','o','^'],
                       4*['none'],
                       [2,3,2,3])
print('minimum band gap/2 : (eV)', Eg/2)


minimum band gap/2 : (eV) 0.26673520062267914


### compute Self-energy and Level-broadning

In [304]:
Sigma_s_comp = np.zeros((M_red,Epts),dtype=complex)
Sigma_d_comp = np.zeros((M_red,Epts),dtype=complex)
Gamma_s_comp = np.zeros((M_red,Epts),dtype=complex)
Gamma_d_comp = np.zeros((M_red,Epts),dtype=complex)

for j in range(0,M_red):
    Sigma_s_comp[j] = gamma**2 * gR_surf_s1[j]
    Sigma_d_comp[j] = gamma**2 * gR_surf_d1[j]
        
    Gamma_s_comp[j] = 1j*(Sigma_s_comp[j] - np.conjugate(Sigma_s_comp[j]))
    Gamma_d_comp[j] = 1j*(Sigma_d_comp[j] - np.conjugate(Sigma_d_comp[j]))

In [305]:
if(PrintFigures):
    custom_plot_single([E,E,E,E], 
                       [ np.real(Sigma_s_comp[0]), np.imag(Sigma_s_comp[0]), np.real(Sigma_d_comp[0]), np.imag(Sigma_d_comp[0])],
                       [r'$\Re(\Sigma_s^{1,1})$',r'$\Im(\Sigma_s^{1,1})$',r'$\Re(\Sigma_d^{N,N})$',r'$\Im(\Sigma_d^{N,N})$'],
                       [0,4],[-3,3],[r'Energy, E / (eV)',r'Component of self-energy, $\Sigma$'],'Sigma.png',
                       ['r','r','b','b'],
                       ['solid','dashed','solid','dashed'],
                       ['o','^','o','^'],
                       4*['none'],
                       [2,3,2,3])
print('minimum band gap/2 : (eV)', Eg/2)


minimum band gap/2 : (eV) 0.26673520062267914


In [306]:
if(PrintFigures):
    custom_plot_single([E,E], 
                       [ np.real(Gamma_s_comp[0]), np.real(Gamma_d_comp[0])],
                       [r'$\Gamma_s^{1,1}$',r'$\Gamma_d^{N,N}$'],
                       [-8,8],[0,5],[r'Energy, E / (eV)',r'Level broadning, $\Gamma$'],'Gamma.png',
                       ['r','b'],
                       ['solid','dashed'],
                       2*[None],
                       2*['none'],
                       [3,3])

In [307]:
from numpy.linalg import inv
zplus=1j*1e-12 #small imaginary number

T=np.zeros(Epts,dtype=float)
Cond=np.zeros(Epts,dtype=float)

D=np.zeros(Epts,dtype=float)

Cond_tot = 0.
I=0. #Current
deltaE=E[1]-E[0]
print(deltaE)

0.00040006000149972465


In [308]:
for e in range(0,Epts):
    Sigma_s=np.zeros((M_red*N,M_red*N),dtype=complex)
    Sigma_d=np.zeros((M_red*N,M_red*N),dtype=complex)
    Gamma_s=np.zeros((M_red*N,M_red*N),dtype=complex)
    Gamma_d=np.zeros((M_red*N,M_red*N),dtype=complex)
    for j in range(0,M_red):
        Sigma_s[j*N][j*N] = Sigma_s_comp[j][e]
        Sigma_d[(j+1)*N-1][(j+1)*N-1] = Sigma_d_comp[j][e]
        Gamma_s[j*N][j*N] = Gamma_s_comp[j][e]
        Gamma_d[(j+1)*N-1][(j+1)*N-1] = Gamma_d_comp[j][e]
#         print('\Sigma_s',Sigma_s[j*N][j*N], Sigma_s_comp[j][e])
#         print('\Sigma_d',Sigma_d[(j+1)*N-1][(j+1)*N-1], Sigma_d_comp[j][e])
#         print('\Gamma_s',Gamma_s[j*N][j*N], Gamma_s_comp[j][e])
#         print('\Gamma_d',Gamma_d[(j+1)*N-1][(j+1)*N-1], Gamma_d_comp[j][e])  

    #or do the following (OK because Sigma matrix is diagonal)
    #Gamma_s = 1j*(Sigma_s - Sigma_s.conj().T) 
    #Gamma_d = 1j*(Sigma_d - Sigma_d.conj().T) 
    #sns.heatmap(np.real(Sigma_d+Sigma_s),cmap='coolwarm',annot=True)
    
    G_R=inv(((E[e]+zplus)*np.eye(M_red*N))- H - Sigma_s - Sigma_d)
    G_A = G_R.conj().T
    T[e]=np.real(np.trace(Gamma_s@G_R@Gamma_d@G_A))
    Cond[e]= -G_quantum*T[e]*dfdE_c[e]
    D[e]=np.real(np.trace(1j*(G_R - G_A)/(2*np.pi)))
    Cond_tot += Cond[e]*deltaE

     
#    I=I+(dE*IE*T[e]*(f1[k]-f2[k])) 

In [309]:
print(Cond_tot)

2.004784054996696e-11


In [310]:
if(PrintFigures):
    custom_plot_single([E], 
                       [T/M_red],
                       [r'$T(E) (%d,%d)$'%(m,n)],
                       [-8,10],[0,1.1],[r'Energy, E / (eV)',r'Transmission probability, T'],'Transmission_Prob_%d_%d.png'%(m,n),
                       ['k'],
                       ['solid'],
                       1*[None],
                       1*['none'],
                       [3])


In [311]:
print(Cond)

[-0.00000000e+000 -0.00000000e+000 -0.00000000e+000 ...  1.41592066e-144
  1.39434209e-144  1.38378776e-144]


In [312]:
if(PrintFigures):
    custom_plot_single([E], 
                       [Cond/G_quantum],
                       [r'$(%d,%d)$'%(m,n)],
                       [-2,2],[0,1e-6],[r'Energy, E / (eV)',r'Integrand of Conductance function, $G/(2q^2/h)$'],'Conductance_%d_%d.png'%(m,n),
                       ['k'],
                       ['solid'],
                       1*[None],
                       1*['none'],
                       [3])


In [313]:
print(D)

[0.95819402 0.9579825  0.95777102 ... 0.95777102 0.9579825  0.95819402]


# Comparison with Mintmire & White (Theory and First principles)

In [314]:
nu0=np.round(2*m/3)
print(nu0+M)
#Mode_arr_Mintmire=np.linspace(-M,M,2*M+1)
Mode_arr_Mintmire=np.linspace(-M,M,2*M+1)

#Mode_arr_datta=np.linspace(nu0-M,nu0+M,2*M+1)
print('Mode_arr_Mintmire',Mode_arr_Mintmire)
#print('Mode_arr_datta',Mode_arr_datta)

Lambda_Mintmire = 2*R/acc
print('Lambda_Mintmire',Lambda_Mintmire)

print('sorted_M_arr',sorted_M_arr)

28.0
Mode_arr_Mintmire [-17. -16. -15. -14. -13. -12. -11. -10.  -9.  -8.  -7.  -6.  -5.  -4.
  -3.  -2.  -1.   0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.
  11.  12.  13.  14.  15.  16.  17.]
Lambda_Mintmire 9.372591222170465
sorted_M_arr [6, 11, 5, 12, 7, 10, 4, 13, 8, 9, 3, 14, 2, 15, 1, 16, 17]


In [315]:
##Francois' book
D_th_lin = np.zeros(Epts, dtype=float)

for e in range(0,Epts):
    const = acc*np.sqrt(3)/(np.pi**2*R*gamma)
    Sum=0.
    for nu in Mode_arr_Mintmire:   
        eps_m = np.abs(3*nu-m+n)*acc*gamma/(2*R) 
        if(np.abs(E[e]) > np.abs(eps_m)):
            Sum += np.abs(E[e])/cmath.sqrt(E[e]**2 - eps_m**2)
        elif(np.abs(E[e]) < np.abs(eps_m)):
            Sum +=0
    D_th_lin[e] = const*np.real(Sum)#*num_atoms

dE_dk = np.zeros((M,np.size(k)),dtype=float)
for p in range(1,M+1):
    dE_dk[p-1] = np.gradient(E_k[p*2-2])
    
# D_th = np.zeros(Epts, dtype=float)   
# for e in range(0,Epts):
#     print('e',e)
#     const = acc**2*np.sqrt(3)/(np.pi**2*R)
#     Sum=0.+1j*0.
#     for p in range(1,M+1):
#         #print(p)
#         k_sum=0.
#         for kx in range(0,np.size(k)):
#             convergence=np.abs((np.abs(E[e])-np.abs(E_k[p*2-2][kx]))/E[e])
#             if(convergence < 1e-3): 
#                 k_sum += 1./(np.abs(dE_dk[p-1][kx]))
#                 if(np.abs(dE_dk[p-1][kx])==0):
#                     print('mode, convergence, E, E(k)', sorted_M_arr[p-1], convergence, E[e],E_k[p*2-2][kx])
#             else:
#                 k_sum=0.
#         Sum += k_sum*(k[1]-k[0])
#     D_th[e] = const*np.real(Sum)


###Supriyo Datta's
#
# D_th_datta = np.zeros(Epts, dtype=float)

# for e in range(0,Epts):
#     a=3*acc/2.
#     L=1*(3/4)*acc/17
#     const = (2*L/(np.pi*a*gamma))
#     Sum=0.
#     for nu in Mode_arr_datta:
#         Ek = (gamma*2*np.pi/np.sqrt(3))*(3*nu/(2*m)-1)+1j*1e-12
#         Sum += E[e]/cmath.sqrt(E[e]**2 - Ek**2)
        
#     D_th_datta[e] = const*Sum

In [316]:
if(PrintFigures):
    custom_plot_single([Lambda_Mintmire*E/gamma,Lambda_Mintmire*E/gamma], 
                       [Lambda_Mintmire*gamma*D*2/M/N,Lambda_Mintmire*gamma*D_th_lin],
                       [r'NEGF [(%d,%d) CNT]'%(m,n), r'Mintmire & White (Theory)'],
                       [-5,5],[0,10],[r'Normalized energy, $\Lambda E$ / $\gamma$',r'Normalized Density of States, $\Lambda \gamma D(E)$'],'DOS_MintmireCompare_%d_%d.png'%(m,n),
                       ['k','r','b'],
                       ['solid','dashed','dotted'],
                       3*[None],
                       3*['none'],
                       3*[3])

In [317]:
def get_bandgap_from_DOS(E, Dnorm):
    Epts = np.size(E)
    e_plus_traverse = True
    e_minus_traverse = True
    e_plus = 0.
    e_minus = 0.
    bandgap = 0.
    if(Dnorm[int(Epts/2.)] > 1e-3):
        bandgap = 0. 
    else:
        for e in range(int(Epts/2.), Epts):
            if(e_plus_traverse):
                if(E[e] > 0.):
                    if(Dnorm[e] > 0.1):
                        e_plus = E[e]
                        e_plus_traverse = False
                        break
                
        for e in np.arange(int(Epts/2.), 0,-1):
            if(e_minus_traverse):
                if(E[e] < 0.):
                    if(Dnorm[e] > 0.1):
                        e_minus = E[e]
                        e_minus_traverse = False
                        break
                        print(E[e],D[e])
        bandgap = e_plus - e_minus
    return bandgap

bandgap_negf = get_bandgap_from_DOS(E, Lambda_Mintmire*gamma*D*2/M/N )
print('m, n, bandgap_negf, bandgap_theory: ', m, n, bandgap_negf, Eg_min)

m, n, bandgap_negf, bandgap_theory:  17 0 0.5428814220355509 0.5334704012453583
